In [2]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
import unicodedata

## Scrapping for Biggest European cities

In [4]:
url = ("https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits") 
response = requests.get(url)
time.sleep(2)
soup = BeautifulSoup(response.content, "html.parser")
parent = soup.find(class_ = 'wikitable sortable')

In [5]:
children = parent.contents[1]

In [256]:
dictionary = {"City": [],"Country":[],"Population":[]}
for i, child in enumerate(children):
    if((i != 0) & (i%2 == 0)):
        dictionary['City'].append(child.contents[3].get_text("|", strip=True).replace('\n',''))
        dictionary['Population'].append(child.contents[7].get_text("|", strip=True))
        dictionary['Country'].append(child.contents[5].get_text("|", strip=True))
    if(i == 40):
        break

In [275]:
table = pd.DataFrame.from_dict(dictionary)
table.Population = pd.to_numeric(table.Population.str.split('|').str[0].str.replace(',',''))
table.City = table.City.str.split('|').str[0]

In [276]:
table

,City,Country,Population
0,Istanbul,Turkey,15462452
1,Moscow,Russia,12195221
2,London,United Kingdom,9126366
3,Saint Petersburg,Russia,5383890
4,Berlin,Germany,3748148
5,Madrid,Spain,3223334
6,Kyiv,Ukraine,2950800
7,Rome,Italy,2844750
8,Bucharest,Romania,2155240
9,Paris,France,2140526


## Scraping for country codes

In [259]:
url = ("https://www.iban.com/country-codes") 
response = requests.get(url)
time.sleep(2)
soup = BeautifulSoup(response.content, "html.parser")

In [260]:
def get_country_codes(soup):
    dictionary = {"Country":[],"Country_code":[]}
    table_html = soup.find('table',id="myTable")
    for row in table_html.find_all('tr'):
        for j, column in enumerate(row.find_all('td')):
            if (j==0):
                dictionary['Country'].append(column.get_text())
            if(j==1):
                dictionary['Country_code'].append(column.get_text())
    table_country_codes = pd.DataFrame.from_dict(dictionary)
    return table_country_codes

In [261]:
table_country_codes = get_country_codes(soup)

In [262]:
table_country_codes

,Country,Country_code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
...,...,...
244,Wallis and Futuna,WF
245,Western Sahara,EH
246,Yemen,YE
247,Zambia,ZM


## Merging cities and country codes

In [277]:
table.Country = table.Country.apply(lambda x:  table_country_codes[table_country_codes.Country.str.contains(x)].Country.values[0])

In [278]:
big_table = table.merge(table_country_codes,how='left')

In [279]:
big_table

,City,Country,Population,Country_code
0,Istanbul,Turkey,15462452,TR
1,Moscow,Russian Federation (the),12195221,RU
2,London,United Kingdom of Great Britain and Northern I...,9126366,GB
3,Saint Petersburg,Russian Federation (the),5383890,RU
4,Berlin,Germany,3748148,DE
5,Madrid,Spain,3223334,ES
6,Kyiv,Ukraine,2950800,UA
7,Rome,Italy,2844750,IT
8,Bucharest,Romania,2155240,RO
9,Paris,France,2140526,FR


## Creating csv file

In [281]:
# big_table.to_csv('data/cities.csv',index=False)